In [ ]:
from PIL import Image, ImageDraw
from skimage import io
from flask import Flask, Response
from kafka import KafkaConsumer
from time import sleep
from json import loads
from datetime import datetime
from neo4j import GraphDatabase

import cv2
import sys
import numpy as np
import argparse
import pickle
import os
import matplotlib.pyplot as plt
import json
import pandas as pd

consumer = KafkaConsumer('testOntology', bootstrap_servers='localhost:9092')

readed = 0

for message in consumer:
    k = message.value
    while True:
        try:
            if(k.decode() == 'start video'):
                readed = 1
                print(str(k.decode()))
            elif(k.decode() == 'end video'):
                readed = 1
                print(str(k.decode()))
            break
        except:
            break
    if(readed == 1):
        readed = 0
        continue
        
    y = np.frombuffer(k, dtype='uint8')    #bytes to int
    img = y.reshape(360, 640, 3) #reshape
    net = cv2.dnn.readNet("yolov7.weights", "yolov7.cfg") #configuration
    classes = []
    with open("coco.names", "r") as f:  #labels
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames() #array of labels

    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    #colors = np.random.uniform(0, 255, size=(len(classes), 3))

    #img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    objects = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            #A Confidence Score is a number between 0 and 1
            #that represents the likelihood that the output of a 
            #Machine Learning model is correct and will satisfy a user's request.
            if confidence > 0.3:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) #give indices for each box

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            print(label)
            objects.append(label)

    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "1234"))
    session = data_base_connection.session()
    if(objects.count("person") >=1 and objects.count("knife") >=1):
        s="merge (v1:Video {name:'video1'})"+\
          "merge (e1:Event {name:'event1'})"+\
          "merge (p1:Person {name: 'person1'})"+\
          "merge (p2:Person {name: 'person2'})"+\
          "merge (k:Knife {name: 'knife'})"+\
          "merge (a:Killing {name: 'killing'})"+\
          "merge (i:Indicator {name: 'Murder'})"+\
          "MERGE (e1)-[:OBTAINED_FROM]->(v1)"+\
          "MERGE (e1)-[:ASSOCIATED_WITH]->(i)"+\
          "MERGE (e1)-[:HAS_OBJECTS]->(p1)"+\
          "MERGE (e1)-[:HAS_OBJECTS]->(p2)"+\
          "MERGE (e1)-[:HAS_OBJECTS]->(k)"+\
          "MERGE (e1)-[:HAS_ACTIONS]->(a)"+\
          "MERGE (a)-[:MADE_BY]->(p1)"+\
          "MERGE (a)-[:MADE_ON]->(p2)"+\
          "merge (x:Video {name:'Video'})"+\
          "merge (y:Event {name:'Criminal Event'})"+\
          "merge (z:Object {name:'Person'})"+\
          "merge (w:Object {name:'Knives'})"+\
          "merge (s:Action {name:'Killing'})"+\
          "merge (i1:Indicator {name:'Indicator'})"+\
          "MERGE (v1)-[:IS_A]->(x)"+\
          "MERGE (e1)-[:IS_A]->(y)"+\
          "MERGE (p1)-[:IS_A]->(z)"+\
          "MERGE (p2)-[:IS_A]->(z)"+\
          "MERGE (k)-[:IS_A]->(w)"+\
          "MERGE (a)-[:IS_A]->(s)"+\
          "MERGE (i)-[:IS_A]->(i1)"
        
        
        session.run(s)